## 4.6.5 Discoveryで類似検索

### Wikipediaからテキストの一覧を作成

In [ ]:
# 必要ライブラリの導入
!pip install wikipedia | tail -n 1

In [ ]:
# 日本百名湯のうち、wikipediaに記事のある温泉のリスト

title_list = ['菅野温泉','養老牛温泉','定山渓温泉','登別温泉','洞爺湖温泉','ニセコ温泉郷','朝日温泉 (北海道)',
          '酸ヶ湯温泉','蔦温泉', '花巻南温泉峡','夏油温泉','須川高原温泉','鳴子温泉郷','遠刈田温泉','峩々温泉',
           '乳頭温泉郷','後生掛温泉','玉川温泉 (秋田県)','秋ノ宮温泉郷','銀山温泉','瀬見温泉','赤倉温泉 (山形県)',
           '東山温泉','飯坂温泉','二岐温泉','那須温泉郷','塩原温泉郷','鬼怒川温泉','奥鬼怒温泉郷',
           '草津温泉','伊香保温泉','四万温泉','法師温泉','箱根温泉','湯河原温泉',
           '越後湯沢温泉','松之山温泉','大牧温泉','山中温泉','山代温泉','粟津温泉',
           '奈良田温泉','西山温泉 (山梨県)','野沢温泉','湯田中温泉','別所温泉','中房温泉','白骨温泉','小谷温泉',
           '下呂温泉','福地温泉','熱海温泉','伊東温泉','修善寺温泉','湯谷温泉 (愛知県)','榊原温泉','木津温泉',
           '有馬温泉','城崎温泉','湯村温泉 (兵庫県)','十津川温泉','南紀白浜温泉','南紀勝浦温泉','湯の峰温泉','龍神温泉',
           '奥津温泉','湯原温泉','三朝温泉','岩井温泉','関金温泉','玉造温泉','有福温泉','温泉津温泉',
           '湯田温泉','長門湯本温泉','祖谷温泉','道後温泉','二日市温泉 (筑紫野市)','嬉野温泉','武雄温泉',
           '雲仙温泉','小浜温泉','黒川温泉','地獄温泉','垂玉温泉','杖立温泉','日奈久温泉',
           '鉄輪温泉','明礬温泉','由布院温泉','川底温泉','長湯温泉','京町温泉',
           '指宿温泉','霧島温泉郷','新川渓谷温泉郷','栗野岳温泉']

# wikipediaの記事の読み取り
# 2.1節参照
import wikipedia
wikipedia.set_lang("ja")

data_list = []
for index, title in enumerate(title_list):
    print(index+1, title)
    text = wikipedia.page(title,auto_suggest=False).content
    item = {
        'app_id': index + 1,
        'title': title,
        'text': text
    }
    data_list.append(item)

## Discoveryにデータ投入

In [ ]:
# 必要ライブラリの導入
!pip install ibm_watson | tail -n 1

In [ ]:
# 資格情報の設定 (個別に設定します)

discovery_credentials = {
  "apikey": "xxxx",
  "iam_apikey_description": "xxxx",
  "iam_apikey_name": "xxxx",
  "iam_role_crn": "xxxx",
  "iam_serviceid_crn": "xxxx",
  "url": "xxxx"
}

In [ ]:
# Discovery APIの初期化

import json
import os
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

version = '2019-04-30'

authenticator = IAMAuthenticator(discovery_credentials['apikey'])
discovery = DiscoveryV1(
    version=version,
    authenticator=authenticator
)
discovery.set_service_url(discovery_credentials['url'])

In [ ]:
# environment_id、collection_id、configuration_id の取得
# すでにUIで1つのprivate collectionが作成済みであることが前提

# environment id の取得
environments = discovery.list_environments().get_result()['environments']
environment_id = environments[0]['environment_id']
if environment_id == 'system':
    environment_id = environments[1]['environment_id']
print('environment_id: ', environment_id)

# collection id の取得
collection_id = discovery.list_collections(environment_id ).get_result()['collections'][0]['collection_id']
print('collection_id: ' , collection_id)

# configuration idの取得
configuration_id = discovery.list_configurations(environment_id).get_result()['configurations'][0]['configuration_id']
print('configuration_id: ', configuration_id)

## 文書のロードと削除用関数 (4.6.2節)

In [ ]:
# 文書ロード関数
# collection_id: 対象コレクション
# sample_data: 書き込み対象テキスト (json形式の配列)
# key_name: 文書のユニークキー名称

def load_text( collection_id, sample_data, key_name):
    for item in sample_data:
        
        # itemごとにワークのjsonファイルを作成
        print(item)
        key = item.get(key_name)
        filename = str(key) + '.json'
        f = open(filename, 'w')
        json.dump(item, f)
        f.close()
        
        # 書き込み可能かのチェック
        collection = discovery.get_collection(environment_id, collection_id).get_result()
        proc_docs = collection['document_counts']['processing']
        while True:
            if proc_docs < 20:
                break
            print('busy. waiting..')
            time.sleep(10)
            collection = discovery.get_collection(environment_id, collection_id)
            proc_docs = collection['document_counts']['processing']

        # jsonファイル名を引数にDiscoveryへデータロード
        with open(filename) as f:
            add_doc = discovery.add_document(environment_id, collection_id, file = f)
        os.remove(filename)

In [ ]:
# 特定のコレクションの全文書を削除する関数
# collection_id: 対象コレクション

def delete_all_docs(collection_id):

    # 文書件数取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection['document_counts']['available']

    results = discovery.query(environment_id, collection_id, return_fields='id', count=doc_count).get_result()["results"]
    ids = [item["id"] for item in results]

    for id in ids:
        print('deleting doc: id =' + id)
        discovery.delete_document(environment_id, collection_id, id)

In [ ]:
# 既存文書の全削除
delete_all_docs(collection_id)

In [ ]:
# wikipedia文書のロード
load_text(collection_id, data_list, 'app_id')

## 類似検索

In [ ]:
# リスト 4.6.14
# 定山渓温泉のid値を調べる

return_fields = 'app_id,title'
filter_text = 'title::定山渓温泉'

query_results = discovery.query(environment_id, collection_id, 
    filter=filter_text, 
    return_fields=return_fields).get_result()[ "results"]

similar_document_id = query_results[0]["id"]
print(similar_document_id)

In [ ]:
# リスト 4.6.15
# 類似検索と結果表示

# 類似検索実施
simular_results = discovery.query(environment_id, collection_id, 
    similar = 'true',
    similar_document_ids = similar_document_id)
res = simular_results.get_result()
res2 = res['results']

# 結果表示
for item in res2:
    metadata = item['result_metadata']
    score = metadata['score']
    app_id = item['app_id']
    title = item['title']
    print(app_id, title, score)